In [1]:
import os
os.chdir("../")
%pwd

'/Users/valentinmonney/Documents/data_science/python/MachineLearningProjects/Kidney-Disease-Classification-Deep-Learning-Project'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [3]:
from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml, create_directories
import tensorflow as tf

2023-10-21 21:16:02.820462: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        """Return the training configuration."""
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = (
            Path(self.config.data_ingestion.unzip_dir)
            / "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"
        )
        create_directories([Path(training.root_dir)])

        return TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
        )

In [4]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [16]:
class Training:
    """Training class for CNN Classifier."""

    def __init__(self, config: TrainingConfig) -> None:
        """Initialize the Training object.

        Args:
        ----
            config (TrainingConfig): The configuration object for training.
        """
        self.config = config

    def get_base_model(self) -> None:
        """Load the updated base model."""
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self) -> None:
        """Train and validate the generator."""
        datagenerator_kwargs = {"rescale": 1.0 / 255, "validation_split": 0.20}

        dataflow_kwargs = {
            "target_size": self.config.params_image_size[:-1],
            "batch_size": self.config.params_batch_size,
            "interpolation": "bilinear",
        }

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs,
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs,
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs,
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs,
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model) -> None:
        """Save the trained model to the specified path.

        Args:
        ----
            path (Path): The path where the trained model will be saved.
            model (tf.keras.Model): The trained model to be saved.
        """
        model.save(path)

    def train(self, callback_list: list = None) -> None:
        """Train the model.

        Args:
        ----
            callback_list (list, optional): List of callbacks to be used during training. Defaults to None.
        """
        if callback_list is None:
            callback_list = []

        self.steps_per_epoch = (
            self.train_generator.samples // self.train_generator.batch_size
        )
        self.validation_steps = (
            self.valid_generator.samples // self.valid_generator.batch_size
        )

        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=10, verbose=1, restore_best_weights=True
        )

        callback_list.append(early_stopping)

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list,
        )

        self.save_model(path=self.config.trained_model_path, model=self.model)

In [17]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise e

[2023-10-21 21:48:46,678: INFO: common] yaml file: config/config.yaml loaded successfully
[2023-10-21 21:48:46,683: INFO: common] yaml file: params.yaml loaded successfully
[2023-10-21 21:48:46,684: INFO: common] created directory at: artifacts
[2023-10-21 21:48:46,685: INFO: common] created directory at: artifacts/training
Found 2487 images belonging to 4 classes.
Found 9959 images belonging to 4 classes.
Epoch 1/30


2023-10-21 21:48:47.837384: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


 32/622 [>.............................] - ETA: 32:50 - loss: 22.2152 - accuracy: 0.3066

KeyboardInterrupt: 